In [23]:
!pip install langchain playwright langchain_community langchain_openai lxml langchainhub

  Using cached tiktoken-0.9.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 4.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.1/599.1 kB 7.1 MB/s eta 0:00:00
Using cached tiktoken-0.9.0-cp312-cp312-macosx_11_0_arm64.whl (1.0 MB)
  Attempting uninstall: greenlet
    Found existing installation: greenlet 3.0.1
    Uninstalling greenlet-3.0.1:
      Successfully uninstalled greenlet-3.0.1


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="cpu"
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
llm = HuggingFacePipeline(pipeline=pipe)


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu
/var/folders/w5/81rp1d0x34nd8g9c4y0lbncc0000gn/T/ipykernel_62240/345718258.py:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [3]:
import nest_asyncio
import asyncio

nest_asyncio.apply()

from langchain_community.tools.playwright.utils import create_async_playwright_browser
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit

async def setup_playwright_tools():
    async_browser = create_async_playwright_browser()
    toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
    return toolkit.get_tools()

tools = await setup_playwright_tools()


In [4]:
from langchain.prompts import PromptTemplate
from langchain.agents import create_react_agent, AgentExecutor

prompt_template = """
You are a helpful browser automation agent. You have access to these tools:
{tools}

Tool names you can use: {tool_names}

When you're given a task, think step-by-step and use the tools.

You MUST use this format:

Thought: what you want to do
Action: tool_name
Action Input: "input to the tool"

Once the task is done:
Final Answer: the result

Begin!

Task: {input}

{agent_scratchpad}
"""

# ✅ Include all required fields for create_react_agent
prompt = PromptTemplate(
    input_variables=["input", "agent_scratchpad", "tools", "tool_names"],
    template=prompt_template
)

# Let LangChain inject tools/tool_names
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)


In [7]:
command = {
    "input": "Go to https://google.com, and search Partha Sai Guttikonda"
}
agent_executor.invoke(command)



> Entering new AgentExecutor chain...


Exception in thread Thread-9 (generate):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.12/site-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/transformers/generation/utils.py", line 2079, in generate
    self._validate_generated_length(generation_config, input_ids_length, has_default_max_length)
  File "/opt/anaconda3/lib/python3.12/site-packages/transformers/generation/utils.py", line 1416, in _validate_generated_length
    raise ValueError(
ValueError: Input length of input_ids is 2110, but `max_length` is set to 2048. This can lead to unexpected behavior. You should consider increasing `max_length` or, better y

Thought: Go to Google
Action: click_element("input[name='q']")
Action Input: "Partha Sai Guttikonda"


Thought: Click on the search button
Action: navigate_browser("https://www.google.com/search?q=Partha+Sai+Guttikonda")


Thought: Wait for the page to load
Action: wait_for_page_load("https://www.google.com/search?q=Partha+Sai+Guttikonda")


Thought: Check if the search result is displayed
Action: check_element_exists("//div[@class='r']/a")
Action Input: "Partha Sai Guttikonda"


Thought: Click on the search result
Action: navigate_browser("https://www.google.com/search?q=Partha+Sai+Guttikonda")


Thought: Wait for the page to load
Action: wait_for_page_load("https://www.google.com/search?q=Partha+Sai+Guttikonda")


Thought: Check if the search result is displayed
Action: check_element_exists("//div[@class='r']/a")
Action Input: "Partha Sai Guttikonda"


Thought: Click on the search result
Action: navigate_browser("https://www.google.com/search?q=Partha+Sai+Guttikonda")


Thought: Wait

Empty: 